In [98]:
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import export_graphviz
import graphviz
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('training_data.csv')

In [3]:
df.head()

,ATT_PASS,ATT_RUSH,CMP,FF,FG,FGA,FL,FPTS,FR,INT,...,home/away,OverUnder,team,spread,ml,ID,Name_y,Abbreviation,Conference,Division
0,30.1,6.3,19.9,0.0,0.0,0.0,0.2,20.6,0.0,0.0,...,away,43.5,Seahawks,-10.5,-500,28,Seattle Seahawks,SEA,NFC,West
1,0.0,11.3,0.0,0.0,0.0,0.0,0.1,9.4,0.0,0.0,...,away,43.5,Seahawks,-10.5,-500,28,Seattle Seahawks,SEA,NFC,West
2,0.0,8.0,0.0,0.0,0.0,0.0,0.1,5.8,0.0,0.0,...,away,43.5,Seahawks,-10.5,-500,28,Seattle Seahawks,SEA,NFC,West
3,0.0,3.4,0.0,0.0,0.0,0.0,0.0,3.7,0.0,0.0,...,away,43.5,Seahawks,-10.5,-500,28,Seattle Seahawks,SEA,NFC,West
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.6,0.0,0.0,...,away,43.5,Seahawks,-10.5,-500,28,Seattle Seahawks,SEA,NFC,West


In [4]:
df.iloc[0]

ATT_PASS                    30.1
ATT_RUSH                     6.3
CMP                         19.9
FF                             0
FG                             0
FGA                            0
FL                           0.2
FPTS                        20.6
FR                             0
INT                            0
INTS                         0.6
PA                             0
Player            Russell Wilson
Pos_x                         QB
REC                            0
SACK                           0
SAFETY                         0
TD                             0
TDS_PASS                     1.9
TDS_REC                        0
TDS_RUSH                     0.2
Team_x                       SEA
Week                           1
XPT                            0
YDS_AGN                        0
YDS_PASS                     245
YDS_REC                        0
YDS_RUSH                    34.9
Year                        2016
GID                         1412
Name_x    

In [34]:
clean_df = df[['Year', 'Week', 'Player', 'team_abbrev',
               'FPTS', 'DK salary', 'DK points', 'home/away',
               'OverUnder', 'spread', 'ml']]
clean_df.dropna(inplace=True)
clean_df.rename(columns={'DK salary': 'DK_salary',
                         'DK points': 'DK_points',
                         'home/away': 'home'}, inplace=True)

/Users/Alex/anaconda/envs/venv/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [35]:
clean_df.head()

,Year,Week,Player,team_abbrev,FPTS,DK_salary,DK_points,home,OverUnder,spread,ml
0,2016,1,Russell Wilson,SEA,20.6,7900.0,13.92,away,43.5,-10.5,-500
1,2016,1,Christine Michael,SEA,9.4,3700.0,9.10,away,43.5,-10.5,-500
2,2016,1,Thomas Rawls,SEA,5.8,6000.0,8.80,away,43.5,-10.5,-500
3,2016,1,C.J. Prosise,SEA,3.7,3900.0,2.10,away,43.5,-10.5,-500
4,2016,1,Doug Baldwin,SEA,9.6,6600.0,24.20,away,43.5,-10.5,-500


In [39]:
results = smf.ols("DK_points ~ FPTS + DK_salary + OverUnder + spread + ml + C(home) + C(Year):C(Week) + C(team_abbrev)", data=clean_df).fit()
# results.summary()
results = smf.ols("DK_points ~ FPTS + DK_salary + OverUnder + spread + ml", data=clean_df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              DK_points   R-squared:                       0.444
Model:                            OLS   Adj. R-squared:                  0.444
Method:                 Least Squares   F-statistic:                     1787.
Date:                Sun, 02 Sep 2018   Prob (F-statistic):               0.00
Time:                        15:21:45   Log-Likelihood:                -36223.
No. Observations:               11187   AIC:                         7.246e+04
Df Residuals:                   11181   BIC:                         7.250e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8990      0.671     -1.340      0.180      -2.214       0.416
FPTS           0.9213      0.019     47.554      0.000       0.883       0.959
DK_salary      0.0008   5.99e-05     13.769      0.000       0.001       0.001
OverUnder      0.0036      0.015      0.247      0.805      -0.025       0.032
spread         0.0096      0.037      0.257      0.798      -0.064       0.083
ml            -0.0003      0.001     -0.437      0.662      -0.002       0.001
==============================================================================
Omnibus:                     2531.563   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6807.190
Skew:                           1.211   Prob(JB):                         0.00
Kurtosis:                       5.956   Cond. No.                     4.88e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.88e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [67]:
X = clean_df[['FPTS', 'DK_salary', 'OverUnder', 'spread', 'ml']]
y = clean_df['DK_points']

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [82]:
ols = LinearRegression()
ols.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [83]:
ols.score(X_test, y_test)

0.42668929384754173

In [69]:
rf = RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=4, random_state=0)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [70]:
rf.feature_importances_

array([0.72840311, 0.1081889 , 0.059003  , 0.03624725, 0.06815773])

In [71]:
rf.score(X_test, y_test)

0.4249015777601698

In [63]:
tree_in_forest = rf.estimators_[0]

In [64]:
graphviz.Source(export_graphviz(tree_in_forest,
                feature_names=X.columns,
                filled=True,
                rounded=True,
                out_file=None)).view()

'Source.gv.pdf'

In [93]:
model = xgb.XGBRegressor(learning_rate=0.1)
model.fit(X_train, y_train)
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, y, cv=kfold)

In [94]:
results

array([0.48680291, 0.45581985, 0.50424265, 0.46833928, 0.4152795 ,
       0.44290341, 0.40764312, 0.4603986 , 0.45807811, 0.417377  ])

In [110]:
results.mean()

0.45168844203636827

In [95]:
xg.score(X_test, y_test)

0.4372685473923953

In [85]:
xg.feature_importances_

array([0.40209267, 0.30792227, 0.09715994, 0.10014948, 0.09267563],
      dtype=float32)

In [115]:
# A parameter grid for XGBoost
params = {
        'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [116]:
model = xgb.XGBRegressor()
kfold = KFold(n_splits=10, random_state=7)
random_search = RandomizedSearchCV(model,
                                   param_distributions=params,
                                   n_iter=100,
                                   n_jobs=4,
                                   cv=kfold,
                                   verbose=1,
                                   random_state=1001 )

In [117]:
random_search.fit(X, y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   18.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   42.8s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  1.6min finished


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=7, shuffle=False),
          error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=100, n_jobs=4,
          param_distributions={'colsample_bytree': [0.6, 0.8, 1.0], 'learning_rate': [0.01, 0.03, 0.05, 0.1], 'min_child_weight': [1, 5, 10], 'subsample': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5], 'gamma': [0.5, 1, 1.5, 2, 5]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [ ]:
ran

In [109]:
random_search.best_score_

0.4512033151863187

In [113]:
random_search.best_score_

0.45311302373971235